In [87]:
import requests
import time
from datetime import date,datetime,timezone
import pandas as pd
import plotly.graph_objects as go

# Settings
finn_key = "btv5stv48v6q7nvmtsjg"
base_url = "https://finnhub.io/api/v1/stock/candle"

In [122]:
# This function pulls and plots daily stock prices for a company ('symbol') over a given date range and overlays technical analysis
def bollinger(symbol, start_date, end_date):
    # Format API call with passed in parameters
    params = {'symbol': symbol,
              'resolution': 'D',
              'from': start_date,
              'to': end_date,
              'token': finn_key}
    response = requests.get(base_url,params=params).json()
    
    # Convert API response to dataframe
    df = pd.DataFrame(response)
    
    # Create date column that is human readable
    df['date'] = [datetime.utcfromtimestamp(d) for d in df['t']] # utcfromtimestamp for accurate date
    # Typical price is the average of the high, low and close
    df['tp'] = (df.h + df.l + df.c) / 3.0
    
    df.set_index(pd.DatetimeIndex(df['date']), inplace=True) # needs datetime as index for rolling offset
    
    # 20 day moving average of the typical price
    df['ma20']= df.tp.rolling('20D').mean() # 20 day offset
    # 20 day standard deviation of the typical price
    df['std20']= df.tp.rolling('20D').std()
    
    # Define upper and lower Bollinger bands as 2 deviations above and below the moving average
    df['upper'] = df['ma20'] + 2*df['std20']
    df['lower'] = df['ma20'] - 2*df['std20']
    print(df)
    
    # Create a plot using plotly with upper and lower Bollinger bands,
    # a plot for the moving average, 
    # a candlestick plot for the stock we're interested in
    fig = go.Figure(data = [
        go.Scatter(x=df['date'],y=df['lower'],line_color='gray',name='Lower Bollinger Band'),
        go.Scatter(x=df['date'],y=df['upper'],line_color='gray',name='Upper Bollinger Band',fill='tonexty'),
        go.Scatter(x=df['date'],y=df['ma20'],line=dict(color='gray',dash='dash'),name='20 Day Moving Average'),
        go.Candlestick(
            x=df['date'],
            open=df['o'],
            high=df['h'],
            low=df['l'],
            close=df['c'],
            name=symbol
        )
    ])
    fig.update_layout(
    title="{} Price Technical Analysis".format(symbol),
    xaxis_title="Date",
    yaxis_title="Price ($)",
#     legend_title="Legend Title",
    )
    # Allow for adjusting the y-axis
    fig.update_yaxes(fixedrange = False)
    return fig

In [123]:
# stocks = ['AMZN','AAPL','MSFT','TSLA']

end_date = 1602979200 # Oct 18, 2020
# start_date = end_date - 2 years
start_date = end_date-2*365*24*3600-86400 # Oct 18, 2018 need to subtract extra day

for stock in stocks:
    fig = bollinger(stock, start_date, end_date)
    
    fig.show()

                      c            h            l            o   s  \
date                                                                 
2018-10-18  1770.719971  1830.150024  1767.869995  1821.489990  ok   
2018-10-19  1764.030029  1809.099976  1753.000000  1785.160034  ok   
2018-10-22  1789.300049  1809.500000  1756.000000  1784.000000  ok   
2018-10-23  1768.699951  1776.339966  1714.000000  1742.239990  ok   
2018-10-24  1664.199951  1777.709961  1656.560059  1773.699951  ok   
...                 ...          ...          ...          ...  ..   
2020-10-12  3442.929932  3496.239990  3339.550049  3349.939941  ok   
2020-10-13  3443.629883  3492.379883  3424.219971  3467.989990  ok   
2020-10-14  3363.709961  3464.879883  3340.000000  3447.000000  ok   
2020-10-15  3338.649902  3355.879883  3280.000000  3292.010010  ok   
2020-10-16  3272.709961  3399.659912  3160.000000  3363.229980  ok   

                     t        v       date           tp         ma20  \
date             

                     c           h           l           o   s           t  \
date                                                                         
2018-10-18   54.009998   54.939999   53.250000   54.470001  ok  1539820800   
2018-10-19   54.830002   55.310001   54.360001   54.520000  ok  1539907200   
2018-10-22   55.160000   55.840000   54.730000   54.950001  ok  1540166400   
2018-10-23   55.680000   55.810001   53.669998   53.959999  ok  1540252800   
2018-10-24   53.770000   56.060001   53.630001   55.650002  ok  1540339200   
...                ...         ...         ...         ...  ..         ...   
2020-10-12  124.400002  125.160004  119.360001  120.019997  ok  1602460800   
2020-10-13  121.099998  125.320000  119.889999  125.320000  ok  1602547200   
2020-10-14  121.190002  122.989998  119.629997  120.919998  ok  1602633600   
2020-10-15  120.709999  121.190002  119.099998  119.129997  ok  1602720000   
2020-10-16  119.019997  121.547997  118.809998  121.279999  ok  

                     c           h           l           o   s           t  \
date                                                                         
2018-10-18  108.500000  110.529999  107.830002  110.099998  ok  1539820800   
2018-10-19  108.660004  110.860001  108.209999  108.930000  ok  1539907200   
2018-10-22  109.629997  110.540001  108.239998  109.320000  ok  1540166400   
2018-10-23  108.099998  108.970001  105.110001  107.769997  ok  1540252800   
2018-10-24  102.320000  108.489998  101.589996  108.410004  ok  1540339200   
...                ...         ...         ...         ...  ..         ...   
2020-10-12  221.399994  223.839996  216.820007  218.710007  ok  1602460800   
2020-10-13  222.860001  225.080002  220.460007  222.300003  ok  1602547200   
2020-10-14  220.860001  224.179993  219.139999  222.979996  ok  1602633600   
2020-10-15  219.660004  220.320007  216.470001  216.800003  ok  1602720000   
2020-10-16  219.660004  222.289993  219.320007  220.149994  ok  

                     c           h           l           o   s           t  \
date                                                                         
2018-10-18   52.779999   54.200001   52.599998   53.860001  ok  1539820800   
2018-10-19   52.000000   53.930000   50.700001   53.480000  ok  1539907200   
2018-10-22   52.189999   52.369999   50.520000   52.139999  ok  1540166400   
2018-10-23   58.830002   59.590000   52.419998   52.770000  ok  1540252800   
2018-10-24   57.700001   60.889999   57.150002   60.209999  ok  1540339200   
...                ...         ...         ...         ...  ..         ...   
2020-10-12  442.299988  448.670013  438.630005  441.839996  ok  1602460800   
2020-10-13  446.649994  448.839996  436.750000  443.309998  ok  1602547200   
2020-10-14  461.299988  465.750000  447.769989  449.739990  ok  1602633600   
2020-10-15  448.880005  456.309998  442.709991  451.149994  ok  1602720000   
2020-10-16  439.670013  455.949890  438.850006  454.440002  ok  